# Simple Example with Steam and Heat Exchangers

This notebook demonstrates how to assemble, initialize, and run a very simple flowsheet using steam and heat exchangers. This breifly demonstrates the basics of setting up a flowsheet.  See the documentation for a more complete description of the framework.

## Imports

Import all the Pyomo and IDAES modules, classes and functions needed for this example. In the imports below the ```delta_temperature_amtd_rule``` is a function that is used for a Pyomo rule to generate an expression for the temperature difference driving force for a heatexchanger. This is used later to overrive the default LMTD calcualtion.  This makes this example slightly more robust when experimenting with different inputs. 

In [1]:
import pyomo.environ as pe    # Frequently used Pyomo classes and functions 
from pyomo.network import Arc # Class for objects used to connect ports, similar to a material stream
from idaes.core import FlowsheetBlock # IDAES flowsheet model class
from idaes.unit_models import Heater, HeatExchanger # Heater and HeatExchanger model classes
from idaes.unit_models.heat_exchanger import delta_temperature_amtd_rule # A delta T rule
from idaes.property_models import Iapws95ParameterBlock # Physical property parameter block class 

## Initialize DMF

Features of the DMF provide easy online access to documentation.  In the cell below initialize the DMF.

### Example Code

```python
from idaes.dmf import magics
%dmf init . create
```

In [2]:
from idaes.dmf import magics
%dmf init . create

Cannot create new configuration at ".": file "config.yaml" exists. Will try without "create" option

*Success!* Using workspace at "."

## Function to Display Material Contitions

This function will create a table of the material state in every state block.  This makes looking at the results of this example a little easier. This is similar to a steam table, but it shows the ports of each side of the steam, and material states internal to the model if those exist and more that just the inlet and outlet.

In [9]:
import pandas as pd
from idaes.core import StateBlockBase

def state_table(m):
    """
    Create a Pandas data frame that shows the material state in
    every state block.
    
    Args:
        m: a Pyomo model containing IDAES StateBlockBase objects
    
    Returns:
        A Pandas dataframe with material states
    """
    head = ["State Block", 
        "Flow (mol/s)",
        "Temperature (K)",
        "Pressure (Pa)",
        "Enthalpy (J/mol)",
        "Vapor Fraction"]
    st = pd.DataFrame(columns=head)
    j = 0
    for c in model.component_objects():
        if isinstance(c, StateBlockBase):
            for i in c:
                row = [c[i].name, 
                       pe.value(c[i].flow_mol), 
                       pe.value(c[i].temperature), 
                       pe.value(c[i].pressure), 
                       pe.value(c[i].enth_mol),
                       pe.value(c[i].vapor_frac)]
                st.loc[j] = row
                j += 1
    return st

## Create a Flowsheet

The first step to creating a running IDAES model is to create a flowsheet obojet. In the code below the following steps should be performed.

* Create and a Pyomo ConcreteModel (called "model")
* Add a steady-state flowsheet object the model (attribute of "model" called "fs") 
* Add a property parameter block to the flowsheet (attribute of "fs" called "properties")

Parameter blocks contain parameters that are (usually) constant and common to all propery methods of the same type. Sharing a common parameter block is more efficent and also easier to modify when doing things like parameter estimation.

When creating the Flowsheet model (or other idaes block model) The default argument takes a dictionary with configuration options for the model.  In this case the "dynamic" options should be set to False to create a steady-state flowsheet.


### Example Code

```python
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(default={"dynamic": False})
model.fs.properties = Iapws95ParameterBlock()
```

In [3]:
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(default={"dynamic": False})
model.fs.properties = Iapws95ParameterBlock()

## Create a Heater and HeatExchanger Model

Next unit models can be added to the flowsheet. Here we need to create a Heater object and a HeatExchanger object called "model.fs.heater" and "model.fs.heat_exchanger".  For the heater only configuration option that needs to be passed is "property_package" which takes a property parameter block, in this case "model.fs.properties".

The heat exchanger model is a little more complicated.  It has two sides and each side can use a different property package. The default LMTD rule is also going to be replaced with the AMTD rule which was imported in the first step.  The heat excahger config dictionary is shown below. The "side_1" and "side_2" keys are each paired with a dicionary for configuring the two ControlVolume0Ds representing the two sides of the heat exchangter.  The "delta_temperature_rule" key is paired with a function that takes a Pyomo block and a time index and return a temperature difference expression.

```python
{
    "delta_temperature_rule":delta_temperature_amtd_rule,
    "side_1":{"property_package": model.fs.properties},
    "side_2":{"property_package": model.fs.properties}
}
```

### Example Code

```python
model.fs.heater = Heater(default={"property_package": model.fs.properties})
model.fs.heat_exchanger = HeatExchanger(default={
        "delta_temperature_rule":delta_temperature_amtd_rule,
        "side_1":{"property_package": model.fs.properties},
        "side_2":{"property_package": model.fs.properties}})
```

In [4]:
model.fs.heater = Heater(default={"property_package": model.fs.properties})
model.fs.heat_exchanger = HeatExchanger(default={
        "delta_temperature_rule":delta_temperature_amtd_rule,
        "side_1":{"property_package": model.fs.properties},
        "side_2":{"property_package": model.fs.properties}})

## Getting Reference Documentation

The DMF provides easy access to IDAES framework reference documentation. For example:
```python
%dmf help HeatExchanger
```
will open a new browser tab showing the HeatExchanger class documentation. 

In [19]:
%dmf help HeatExchanger

2018-12-12 12:58:12,602 [INFO] idaes.dmf.help: find HTML docs for module=idaes.unit_models.heat_exchanger class=HeatExchanger on paths=['/home/jeslick/git/idaes/docs/build/html']


## Fix Inlets and Initialize

Fix the inlets of the heater and heat exchanger and the heat duty for the heater. Initialize the units.

The heater model has the has one inlet called "inlet" and a heat duty called "heat_duty" that can be fixed resulting in 0 degrees of freedom for the model. The inlet and the heat duty are indexed by time, but since this is a steady state model, only time zero exists.  The state variables for the inlet are "enth_mol", "flow_mol", and "pressure".  These state varaibles depend on the physical property pacakage.  In this case a version of the IAPWS-95 properties for steam is being used, which uses enthalpy and pressure as state varaibles. 

Set the heater inlet to: molar enthalpy = 4000 J/mol, molar flow = 100 mol/s, and pressure = 101325 Pa.  Set the heater heat duty to 2000000 J/s.

The heat exchanger has two inlets "inlet_1" for "side_1" and "inlet_2" for "side_2".  Fix the inlets as shown in the table.

| variable    | inlet_1    | inlet_2   |
|-------------|------------|-----------|
| enth_mole   | 4000       | 3000      |
| pressure    | 101325     | 101325    |
| flow_mol    | 100        | 100       |

Each unit model has an initilialize method to get the model to a solvable state. Call the inititialize methods for each unit.

### Example Heater Code

```python
# Fix both heat exchanger intlets and initialize
model.fs.heater.inlet[0].enth_mol.fix(4000)
model.fs.heater.inlet[0].flow_mol.fix(100)
model.fs.heater.inlet[0].pressure.fix(101325)
model.fs.heater.heat_duty[0].fix(2000000)

model.fs.heater.initialize()
```

### Example Heat Exchanger Code

```python
# Fix both heat exchanger intlets and initialize
model.fs.heat_exchanger.inlet_1[0].flow_mol.fix(100)
model.fs.heat_exchanger.inlet_1[0].pressure.fix(101325)
model.fs.heat_exchanger.inlet_1[0].enth_mol.fix(4000)
model.fs.heat_exchanger.inlet_2[0].flow_mol.fix(100)
model.fs.heat_exchanger.inlet_2[0].pressure.fix(101325)
model.fs.heat_exchanger.inlet_2[0].enth_mol.fix(3000)

model.fs.heat_exchanger.initialize()
```

In [5]:
# Fix heater inlets and heat duty then initialize unit
model.fs.heater.inlet[0].enth_mol.fix(4000)
model.fs.heater.inlet[0].flow_mol.fix(100)
model.fs.heater.inlet[0].pressure.fix(101325)
model.fs.heater.heat_duty[0].fix(2000000)

model.fs.heater.initialize()

In [6]:
# Fix both heat exchanger intlets and initialize
model.fs.heat_exchanger.inlet_1[0].flow_mol.fix(100)
model.fs.heat_exchanger.inlet_1[0].pressure.fix(101325)
model.fs.heat_exchanger.inlet_1[0].enth_mol.fix(4000)
model.fs.heat_exchanger.inlet_2[0].flow_mol.fix(100)
model.fs.heat_exchanger.inlet_2[0].pressure.fix(101325)
model.fs.heat_exchanger.inlet_2[0].enth_mol.fix(3000)

model.fs.heat_exchanger.initialize()

## Solve the Disconnected Flowsheet

The next step in this example is to make sure the disconnected flowsheet will solve.  To solve the model create a slover the solve like a normal Pyomo model.

### Example Code
```python
solver = pe.SolverFactory('ipopt')
solver.solve(model, tee=True)
```

In [7]:
solver = pe.SolverFactory('ipopt')

In [8]:
solver.solve(model, tee=True)

Ipopt 3.12.11: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.11, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:       21
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:       11
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       11
Total number of in

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 11, 'Number of variables': 11, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.11\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.06775021553039551}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

## Show States

Call the ```state_table(model)``` function written earlier to show a table of results. 

In [10]:
state_table(model)

,State Block,Flow (mol/s),Temperature (K),Pressure (Pa),Enthalpy (J/mol),Vapor Fraction
0,fs.heater.control_volume.properties_in[0.0],100,326.166708,101325,4000,0.000000
1,fs.heater.control_volume.properties_out[0.0],100,373.124296,101325,24000,0.404678
2,fs.heat_exchanger.side_1.properties_in[0.0],100,326.166708,101325,4000,0.000000
3,fs.heat_exchanger.side_1.properties_out[0.0],100,313.819218,101325,3070.04,0.000000
4,fs.heat_exchanger.side_2.properties_in[0.0],100,312.888963,101325,3000,0.000000
5,fs.heat_exchanger.side_2.properties_out[0.0],100,325.237048,101325,3929.96,0.000000


## Conncect the Flowsheet

The next step is to connect the outlet of the heater to the side_1 inlet of the heatexchanger.  First infix the varialbes in ```model.fs.heat_exchanger.inlet_1```. Also increase the flow rate of through side_2 of the heat exchanger to 600 mol/s.

Use a Pyomo Arc to connect the heater outlet to heatexchanger inlet_1 and apply the transforamtion to axpand the arc to a set of constraints.

### Example Code

```python
model.fs.heat_exchanger.inlet_1[0].flow_mol.unfix()
model.fs.heat_exchanger.inlet_1[0].pressure.unfix()
model.fs.heat_exchanger.inlet_1[0].enth_mol.unfix()

model.fs.heat_exchanger.inlet_2[0].flow_mol.fix(600)

model.fs.stream_1 = Arc(source=model.fs.heater.outlet[0], 
                        destination=model.fs.heat_exchanger.inlet_1[0])

pe.TransformationFactory("network.expand_arcs").apply_to(model)

```

In [11]:
model.fs.heat_exchanger.inlet_1[0].flow_mol.unfix()
model.fs.heat_exchanger.inlet_1[0].pressure.unfix()
model.fs.heat_exchanger.inlet_1[0].enth_mol.unfix()

model.fs.heat_exchanger.inlet_2[0].flow_mol.fix(600)

model.fs.stream_1 = Arc(source=model.fs.heater.outlet[0], 
                        destination=model.fs.heat_exchanger.inlet_1[0])

pe.TransformationFactory("network.expand_arcs").apply_to(model)

[None]

## Solve the Connected Model

Solve the model again, and show results.

### Example Code

```python
solver.solve(model, tee=True)
state_table(model)
```

In [20]:
solver.solve(model, tee=True)
state_table(model)

Ipopt 3.12.11: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.11, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:       33
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       13

Total number of variables............................:       14
                     variables with only lower bounds:        8
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       14
Total number of in

,State Block,Flow (mol/s),Temperature (K),Pressure (Pa),Enthalpy (J/mol),Vapor Fraction
0,fs.heater.control_volume.properties_in[0.0],100,326.166708,101325,4000,0.000000
1,fs.heater.control_volume.properties_out[0.0],100,373.124296,101325,24000,0.404678
2,fs.heat_exchanger.side_1.properties_in[0.0],100,373.124296,101325,24000,0.404678
3,fs.heat_exchanger.side_1.properties_out[0.0],100,334.275568,101325,4611.18,0.000000
4,fs.heat_exchanger.side_2.properties_in[0.0],600,312.888963,101325,3000,0.000000
5,fs.heat_exchanger.side_2.properties_out[0.0],600,355.733262,101325,6231.47,0.000000


,State Block,Flow (mol/s),Temperature (K),Pressure (Pa),Enthalpy (J/mol),Vapor Fraction
0,fs.heater.control_volume.properties_in[0.0],100,326.166708,101325,4000,0.000000
1,fs.heater.control_volume.properties_out[0.0],100,373.124296,101325,24000,0.404678
2,fs.heat_exchanger.side_1.properties_in[0.0],100,373.124296,101325,24000,0.404678
3,fs.heat_exchanger.side_1.properties_out[0.0],100,334.275568,101325,4611.18,0.000000
4,fs.heat_exchanger.side_2.properties_in[0.0],600,312.888963,101325,3000,0.000000
5,fs.heat_exchanger.side_2.properties_out[0.0],600,355.733262,101325,6231.47,0.000000


In [16]:
%dmf help HeatExchanger

2018-12-12 12:47:57,369 [INFO] idaes.dmf.help: find HTML docs for module=idaes.unit_models.heat_exchanger class=HeatExchanger on paths=['/home/jeslick/git/idaes/docs/build/html']
